In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from rtsvg import *
from math import sin, cos, pi, sqrt
rt = RACETrack()

In [ ]:
#
# __genericArc__()
#
def __genericArc__(cx, cy, a0, a1, r_inner, r_outer):
    _fn_ = lambda a,r: (cx + r * cos(a * pi / 180.0), cy + r * sin(a * pi / 180.0))
    x0_out,  y0_out  = _fn_(a0, r_outer)
    x0_in,   y0_in   = _fn_(a0, r_inner)
    x1_out,  y1_out  = _fn_(a1, r_outer)
    x1_in,   y1_in   = _fn_(a1, r_inner)
    large_arc = 0 if (a1-a0) <= 180.0 else 1
    _path_ = f'M {x0_out} {y0_out} A {r_outer} {r_outer} 0 {large_arc} 1 {x1_out} {y1_out} L {x1_in} {y1_in} ' + \
                                f' A {r_inner} {r_inner} 0 {large_arc} 0 {x0_in}  {y0_in}  Z'
    return _path_
svg = ['<svg width="500" height="500" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#ffffff" />')
svg.append('<circle cx="250" cy="250" r="100" stroke="black" fill="none" />')
svg.append(f'<path d="{__genericArc__(250, 250,  0,   90, 100, 110)}" stroke="none" stroke-width="1" fill="blue"  />')
svg.append(f'<path d="{__genericArc__(250, 250, 90,  180, 110, 120)}" stroke="none" stroke-width="1" fill="red"   />')
svg.append(f'<path d="{__genericArc__(250, 250, 180, 360, 120, 130)}" stroke="none" stroke-width="1" fill="green" />')
svg.append(f'<path d="{__genericArc__(250, 250, 300,  60,  90, 100)}" stroke="black" stroke-width="1" fill="white" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# - assumes that df has already been grouped -- i.e., no duplicate nbors
#
def concentricGlyph(df, cx, cy, r_inner, r_outer, order=None, nbor='__nbor__', count_by='__count__', count_by_set=False):
    if order is None: order = rt.colorRenderOrder(df, nbor, count_by, count_by_set)
    order = order.with_row_index('__sorter__')
    df    = df.join(order.drop(['count']), left_on=nbor, right_on='index').sort('__sorter__')
    total = df[count_by].sum()
    df    = df.with_columns((360.0 * pl.col(count_by)/total).alias('__angle__')).filter(pl.col('__angle__') > 10.0)
    # df.with_columns(pl.col('__angle__').cum_sum().alias('__cumsum__'))
    angle = 270.0 # start at the top... like a clock...
    svgs  = []
    for i in range(len(df)):
        angle_to = angle + df['__angle__'][i]
        _color_  = rt.co_mgr.getColor(df[nbor][i])
        _path_   = __genericArc__(cx, cy, angle, angle_to, r_inner, r_outer)
        svgs.append(f'<path d="{_path_}" stroke="white" stroke-width="1" fill="{_color_}" />')
        angle    = angle_to
    return svgs

#
# Converted from the following description
# https://stackoverflow.com/questions/5736398/how-to-calculate-the-svg-path-for-an-arc-of-a-circle
#
def polarToCartesian(cx, cy, r, deg):
    rads = (deg-90) * pi / 180.0
    return cx + (r*cos(rads)), cy + (r*sin(rads))
def arcPath(cx, cy, r, deg0, deg1):
    x0,y0 = polarToCartesian(cx,cy,r,deg1)
    x1,y1 = polarToCartesian(cx,cy,r,deg0)
    if (deg1 - deg0) <= 180.0: flag = "0"
    else:                      flag = "1"
    return [f'<path d="M {cx} {cy} L {x0} {y0} A {r} {r} 0 {flag} 0 {x1} {y1}" fill="black" stroke="black" stroke-width="2" />']


_lu_ = {'__count__':[ 5,    10,   15,   12,   1,    10],
        '__dir__'  :['fm', 'to', 'fm', 'to', 'fm', 'to'],
        '__nbor__' :['a',  'a',  'b',  'b',  'c',  'c']}
_order_ = ['b','a','c']

rt.co_mgr.str_to_color_lu['a'] = '#ff0000'
rt.co_mgr.str_to_color_lu['b'] = '#03ac13'
rt.co_mgr.str_to_color_lu['c'] = '#0000ff'

df      = pl.DataFrame(_lu_)
_order_ = rt.colorRenderOrder(df, '__nbor__', '__count__', False)

svg = ['<svg width="1000" height="500" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="1000" height="500" fill="#ffffff" />')

r = 3
for x in range(50, 1000, 75):
    for y in range(75, 500, 75):
        _bar_w_ = y/50
        svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'to'), x, y, r,           r+1*_bar_w_, _order_))
        svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'fm'), x, y, r + _bar_w_, r+2*_bar_w_, _order_))
        svg.extend(arcPath(x, y, r-2, 0, 135))

        r += 0.2

# r == 3 + 27*0.2 --> 3 + 33*0.2 , _bar_w_ = 225/50 = 4.5

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = ['<svg width="1000" height="100" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="1000" height="500" fill="#ffffff" />')

r_min, r_max, _bar_w_ = 7, 15, 4.5
for x in range(50, 1000, 75):
    r = r_min + (r_max - r_min) * x/1000 # results in min of 7.4 and max of 14.6
    svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'to'), x, 50, r,           r+1*_bar_w_, _order_))
    svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'fm'), x, 50, r + _bar_w_, r+2*_bar_w_, _order_))
    svg.extend(arcPath(x, 50, r-2, 0, 135))

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = ['<svg width="1000" height="500" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="1000" height="500" fill="#ffffff" />')
r = 3
for x in range(50, 1000, 75):
    for y in range(75, 500, 75):
        _bar_w_ = y/50
        svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'to'), x, y, r,           r+1*_bar_w_, _order_))
        svg.extend(arcPath(x, y, r-2, 0, 135))
        r += 0.2
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = ['<svg width="1000" height="100" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="1000" height="100" fill="#ffffff" />')
r_min, r_max, _bar_w_ = 7, 15, 4.5
for x in range(50, 1000, 75):
    r = r_min + (r_max - r_min) * x/1000 # results in a min of 7.4 and max of 14.6
    svg.extend(concentricGlyph(df.filter(pl.col('__dir__') == 'to'), x, 50, r,           r+1*_bar_w_, _order_))
    svg.extend(arcPath(x, 50, r-2, 0, 135))
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# After integrating the code above (w/ some renaming) into the rtsvg module
#
svg = ['<svg width="1000" height="100" xmlns="http://www.w3.org/2000/svg">']
svg.append('<rect x="0" y="0" width="1000" height="100" fill="#ffffff" />')
svg.append('<rect x="335" y="0" width="300" height="100" fill="#a0a0a0" />')
svg.append('<line x1="650" y1="45" x2="950" y2="45" stroke="black" stroke-width="2" />')

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 50,  50, 0.0, 0.5,  order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 85,  50, 0.5, 0.75, order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 125, 50, 1.0, 1.0,  order=_order_))

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 200, 50, 0.0, 0.0,  df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 240, 50, 0.5, 0.33, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 290, 50, 1.0, 0.66, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+50,  50, 0.0, 0.5,  order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+85,  50, 0.5, 0.75, order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+125, 50, 1.0, 1.0,  order=_order_))

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+200, 50, 0.0, 0.0,  df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+240, 50, 0.5, 0.33, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 310+290, 50, 1.0, 0.66, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+50,  50, 0.0, 0.5,  order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+85,  50, 0.5, 0.75, order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+125, 50, 1.0, 1.0,  pie_color="#ff0000", order=_order_))

svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+200, 50, 0.0, 0.0,  df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+240, 50, 0.5, 0.33, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))
svg.append(rt.concentricGlyph(df.filter(pl.col('__dir__') == 'to'), 620+290, 50, 1.0, 0.66, df_outer=df.filter(pl.col('__dir__') == 'fm'), order=_order_))

svg.append('</svg>')
rt.tile([''.join(svg)])